# Wersja v11: Model Hierarchiczny z 3 Głowicami i Post-processingiem

## Część 1: Przygotowanie Danych i Inżynieria Cech
Ta część jest odpowiedzialna za wczytanie, oczyszczenie i przetworzenie danych w celu przygotowania ich do treningu modelu. Zapiszemy również wszystkie niezbędne artefakty (mapy, transformatory) na dysku.

In [1]:
import pandas as pd
import numpy as np
import os
import joblib
import gc
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

# Definicja stałych globalnych
ARTIFACTS_DIR = 'artifacts_v11'
PROCESSED_DATA_DIR = 'processed_data_v11'
MAX_TEXT_FEATURES = 20000
RANDOM_STATE = 42

# Tworzenie katalogów, jeśli nie istnieją
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

print(f"Katalogi '{ARTIFACTS_DIR}' i '{PROCESSED_DATA_DIR}' są gotowe.")

Katalogi 'artifacts_v11' i 'processed_data_v11' są gotowe.


In [2]:
def load_location_data(location_path='lokalizacja.csv'):
    print(f"Wczytywanie pliku: {location_path}")
    df_loc = pd.read_csv(
        location_path,
        na_values=['\\N', 'NULL'],
        sep=',',
        header=None,
        names=['id', 'parent_id', 'name', 'type', 'full_name']
    )
    return df_loc

def load_offers_data(offers_path='saleflats_2024_2025.csv'):
    print(f"Wczytywanie pliku: {offers_path}")
    column_names = ['title', 'description', 'area', 'price', 'locationPath']
    try:
        first_row = pd.read_csv(offers_path, header=None, sep=',', nrows=1, on_bad_lines='skip')
        last_col_index = first_row.shape[1] - 1
        usecols = [3, 4, 5, 6, last_col_index]
        df_offers = pd.read_csv(offers_path, header=None, sep=',', usecols=usecols, on_bad_lines='skip')
        df_offers.columns = column_names
    except Exception as e:
        print(f"Wystąpił błąd: {e}")
        df_offers = pd.DataFrame(columns=column_names)
    return df_offers

df_loc = load_location_data()
df_offers = load_offers_data()

# Stworzenie i zapisanie map
id_to_name = dict(zip(df_loc['id'], df_loc['name']))
joblib.dump(id_to_name, os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))
hierarchy_map = dict(zip(df_loc['id'], df_loc['parent_id']))
joblib.dump(hierarchy_map, os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
print("Mapy 'id_to_name' i 'hierarchy_map' zostały stworzone i zapisane.")

Wczytywanie pliku: lokalizacja.csv
Wczytywanie pliku: saleflats_2024_2025.csv
Mapy 'id_to_name' i 'hierarchy_map' zostały stworzone i zapisane.


## Inżynieria Cech i Definicja Zmiennej Celu (REKOMENDACJA #1)

**Zmiana:** Zamiast jednej zmiennej `target_district_id`, tworzymy teraz **trzy oddzielne zmienne docelowe**: `target_district_id`, `target_subdistrict_id` oraz `target_street_id`. Dzięki temu model będzie uczył się przewidywać każdy poziom hierarchii osobno.

In [3]:
# Cechy Numeryczne
df_offers['area'] = pd.to_numeric(df_offers['area'], errors='coerce')
df_offers['price'] = pd.to_numeric(df_offers['price'], errors='coerce')
df_offers.dropna(subset=['area', 'price'], inplace=True)
df_offers = df_offers[df_offers['area'] > 0]
df_offers['price_per_meter'] = df_offers['price'] / df_offers['area']

# NOWA DEFINICJA CELÓW: Dzielnica, Pod-dzielnica, Ulica
path_cols = ['woj_id', 'pow_id', 'gmi_id', 'city_id', 'district_id', 'subdistrict_id', 'street_id']
path_df = df_offers['locationPath'].str.split(',', expand=True)
path_df = path_df.iloc[:, :len(path_cols)] # Zabezpieczenie przed zbyt krótkimi ścieżkami
path_df.columns = path_cols[:path_df.shape[1]]
path_df = path_df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
df_offers = pd.concat([df_offers.reset_index(drop=True), path_df.reset_index(drop=True)], axis=1)

# Finalne czyszczenie i tworzenie df_processed
df_offers.dropna(subset=['city_id'], inplace=True)
df_offers = df_offers[df_offers['city_id'] != 0]

df_processed = df_offers[[
    'title', 'description', 'area', 'price', 'price_per_meter', 'city_id', 
    'district_id', 'subdistrict_id', 'street_id'
]].copy()

df_processed.rename(columns={
    'district_id': 'target_district_id',
    'subdistrict_id': 'target_subdistrict_id',
    'street_id': 'target_street_id'
}, inplace=True)

df_processed['text_features'] = df_processed['title'].fillna('') + " " + df_processed['description'].fillna('')
df_processed.drop(columns=['title', 'description'], inplace=True)
df_processed.dropna(inplace=True)

print("\n--- Nagłówek finalnego df_processed ---\n")
print(df_processed.head())


--- Nagłówek finalnego df_processed ---

    area     price  price_per_meter  city_id  target_district_id  \
0  73.00  766500.0     10500.000000      352                   0   
1  64.80  540000.0      8333.333333      352                   0   
2  51.00  540000.0     10588.235294      352                   0   
3  67.62  544000.0      8044.957113      352                   0   
4  48.00  459000.0      9562.500000      352                   0   

   target_subdistrict_id  target_street_id  \
0                 103786                 0   
1                  99764                 0   
2                  74375                 0   
3                  74375            517513   
4                  95559                 0   

                                       text_features  
0  Mieszkanie trzypokojowe na sprzedaż Mieszkanie...  
1  Sprzedam mieszkanie na parterze 64.8m2 Białyst...  
2  Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki ...  
3  Mieszkanie trzypokojowe na sprzedaż ***Oferta ...

In [4]:
# Sprawdzenie liczności klas dla stratyfikacji
city_counts = df_processed['city_id'].value_counts()
valid_cities = city_counts[city_counts > 1].index
df_filtered = df_processed[df_processed['city_id'].isin(valid_cities)].copy()

# Podział na zbiory (X i y)
target_columns = ['target_district_id', 'target_subdistrict_id', 'target_street_id']
X = df_filtered.drop(columns=target_columns)
y = df_filtered[target_columns]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=X['city_id']
)
print(f"Podział danych: {len(X_train)} próbek treningowych, {len(X_val)} próbek walidacyjnych.")

# Przetwarzanie Cech Numerycznych
numeric_features = ['area', 'price', 'price_per_meter']
imputer = SimpleImputer(strategy='median'); scaler = StandardScaler()
X_train_num = scaler.fit_transform(imputer.fit_transform(X_train[numeric_features]))
X_val_num = scaler.transform(imputer.transform(X_val[numeric_features]))
joblib.dump(imputer, os.path.join(ARTIFACTS_DIR, 'imputer.joblib'))
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))

# Przetwarzanie Cech Tekstowych
vectorizer = TfidfVectorizer(max_features=MAX_TEXT_FEATURES, ngram_range=(1, 2))
X_train_text = vectorizer.fit_transform(X_train['text_features'])
X_val_text = vectorizer.transform(X_val['text_features'])
joblib.dump(vectorizer, os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))

# Przygotowanie pozostałych cech
X_train_city = X_train['city_id'].values
X_val_city = X_val['city_id'].values

# Przygotowanie ODDZIELNYCH celów dla każdej głowicy
y_district_train = y_train['target_district_id'].values; y_district_val = y_val['target_district_id'].values
y_subdistrict_train = y_train['target_subdistrict_id'].values; y_subdistrict_val = y_val['target_subdistrict_id'].values
y_street_train = y_train['target_street_id'].values; y_street_val = y_val['target_street_id'].values

print("Transformatory dopasowane i zapisane.")

Podział danych: 887572 próbek treningowych, 221893 próbek walidacyjnych.
Transformatory dopasowane i zapisane.


In [5]:
# Zapisanie wszystkich przetworzonych danych
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'), X_train_text)
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'), X_val_text)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'), X_train_num)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'), X_val_num)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'), X_train_city)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'), X_val_city)

# Zapis nowych, potrójnych zmiennych docelowych
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_district_train.npy'), y_district_train)
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_subdistrict_train.npy'), y_subdistrict_train)
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_street_train.npy'), y_street_train)
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_district_val.npy'), y_district_val)
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_subdistrict_val.npy'), y_subdistrict_val)
np.save(os.path.join(PROCESSED_DATA_DIR, 'y_street_val.npy'), y_street_val)

print("Wszystkie przetworzone zbiory danych zostały zapisane.")

# Zwolnienie pamięci
del df_loc, df_offers, df_processed, X, y, X_train, X_val, y_train, y_val, df_filtered
del X_train_text, X_val_text, X_train_num, X_val_num, X_train_city, X_val_city
gc.collect()

print("\n" + "="*80)
print("CZĘŚĆ 1 ZAKOŃCZONA POWODZENIEM. ZRESTARTUJ KERNEL.")
print("="*80)

Wszystkie przetworzone zbiory danych zostały zapisane.

CZĘŚĆ 1 ZAKOŃCZONA POWODZENIEM. ZRESTARTUJ KERNEL.


# Część 2: Budowa, Trening i Inferencia Modelu

In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import gc
import scipy.sparse
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import register_keras_serializable

# Ponowna definicja stałych
ARTIFACTS_DIR = 'artifacts_v11'
PROCESSED_DATA_DIR = 'processed_data_v11'
RANDOM_STATE = 42
MAX_TEXT_FEATURES = 20000
MODEL_PATH = os.path.join(ARTIFACTS_DIR, 'best_location_model_v11.keras')

In [2]:
# Wczytanie artefaktów
print("Wczytywanie artefaktów...")
hierarchy_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
id_to_name = joblib.load(os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))

# Wczytanie przetworzonych danych
print("Wczytywanie przetworzonych danych...")
X_train_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'))
X_val_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'))
X_train_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'))
X_val_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'))
X_train_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'))
X_val_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'))
y_district_train = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_district_train.npy'))
y_subdistrict_train = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_subdistrict_train.npy'))
y_street_train = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_street_train.npy'))
y_district_val = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_district_val.npy'))
y_subdistrict_val = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_subdistrict_val.npy'))
y_street_val = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_street_val.npy'))
print("Dane wczytane.")

Wczytywanie artefaktów...
Wczytywanie przetworzonych danych...
Dane wczytane.


## Hierarchiczna Funkcja Straty (REKOMENDACJA #3)

**Zmiana:** Pozostawiamy tylko jedną, poprawną definicję klasy `HierarchicalLoss`. Naprawiono również błąd w metodzie `call` i zapewniono poprawną serializację (konwersja kluczy na `int` w `__init__`).

In [3]:
@register_keras_serializable()
class HierarchicalLoss(tf.keras.losses.Loss):
    def __init__(self, id_to_parent_map, penalty_config, **kwargs):
        super().__init__(**kwargs)
        self.id_to_parent_map = id_to_parent_map
        self.penalty_config = penalty_config
        self.base_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.penalty_wrong_parent = penalty_config.get('wrong_parent', 1.5)
        
        # Poprawka: Konwersja kluczy na int, aby uniknąć błędu podczas wczytywania modelu
        keys = [int(k) for k in id_to_parent_map.keys()]
        values = [int(v) if pd.notna(v) else 0 for v in id_to_parent_map.values()]
        self.parent_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(keys, values, key_dtype=tf.int64, value_dtype=tf.int64), 0
        )

    def call(self, y_true, y_pred):
        y_true_tensor = tf.cast(y_true, dtype=tf.int64)
        y_pred_ids = tf.cast(tf.argmax(y_pred, axis=-1), dtype=tf.int64)
        base_loss = self.base_loss_fn(y_true_tensor, y_pred)
        
        # Ignorujemy straty dla etykiety 'Brak' (ID=0)
        mask = tf.not_equal(y_true_tensor, 0)
        
        true_parents = self.parent_table.lookup(y_true_tensor)
        pred_parents = self.parent_table.lookup(y_pred_ids)
        
        # Kara jest stosowana tylko tam, gdzie etykieta nie jest 'Brak'
        is_correct = tf.equal(y_true_tensor, y_pred_ids)
        is_parent_correct = tf.equal(true_parents, pred_parents)
        
        penalties = tf.where(is_correct, 1.0, self.penalty_wrong_parent) # Kara za złą predykcję
        penalties = tf.where(is_parent_correct, 1.0, penalties) # Jeśli rodzic jest ok, nie ma kary - to można zmienić
        
        penalized_loss = base_loss * penalties
        masked_loss = tf.where(mask, penalized_loss, 0.0)

        return tf.reduce_sum(masked_loss) / tf.reduce_sum(tf.cast(mask, tf.float32))

    def get_config(self):
        base_config = super().get_config()
        config = {"id_to_parent_map": self.id_to_parent_map, "penalty_config": self.penalty_config}
        return {**base_config, **config}

print("Hierarchiczna funkcja straty zdefiniowana i gotowa do użycia.")

Hierarchiczna funkcja straty zdefiniowana i gotowa do użycia.


## Definicja Architektury Modelu (3 Głowice)

**Zmiana:** Budujemy model Keras z trzema oddzielnymi wyjściami: `district_output`, `subdistrict_output` i `street_output`. Pozwoli to na jednoczesne przewidywanie wszystkich poziomów lokalizacji.

In [4]:
def create_mapper(y_train, y_val):
    """Tworzy mapowanie z oryginalnych ID na ciągłe indeksy."""
    unique_ids = np.unique(np.concatenate([y_train, y_val]))
    mapper = {val: i for i, val in enumerate(unique_ids)}
    return mapper

def map_labels(y, mapper):
    """Mapuje etykiety używając stworzonego mapowania."""
    return np.array([mapper.get(val, 0) for val in y])

# Mapowanie dla każdej głowicy
district_map = create_mapper(y_district_train, y_district_val)
subdistrict_map = create_mapper(y_subdistrict_train, y_subdistrict_val)
street_map = create_mapper(y_street_train, y_street_val)
city_map = create_mapper(X_train_city, X_val_city)

y_district_train_mapped = map_labels(y_district_train, district_map)
y_district_val_mapped = map_labels(y_district_val, district_map)
y_subdistrict_train_mapped = map_labels(y_subdistrict_train, subdistrict_map)
y_subdistrict_val_mapped = map_labels(y_subdistrict_val, subdistrict_map)
y_street_train_mapped = map_labels(y_street_train, street_map)
y_street_val_mapped = map_labels(y_street_val, street_map)
X_train_city_mapped = map_labels(X_train_city, city_map)
X_val_city_mapped = map_labels(X_val_city, city_map)

joblib.dump(city_map, os.path.join(ARTIFACTS_DIR, 'city_map.joblib'))

# Definicja parametrów
NUM_DISTRICTS = len(district_map)
NUM_SUBDISTRICTS = len(subdistrict_map)
NUM_STREETS = len(street_map)
NUM_CITIES = len(city_map)
NUM_FEATURES = X_train_num.shape[1]

# Definicja modelu
input_text = Input(shape=(MAX_TEXT_FEATURES,), name='text_input', sparse=True)
input_num = Input(shape=(NUM_FEATURES,), name='num_input')
input_city = Input(shape=(1,), name='city_input')

# Wspólna część modelu (ciało)
text_branch = Dense(128, activation='relu')(input_text); text_branch = Dropout(0.3)(text_branch)
num_branch = Dense(64, activation='relu')(input_num); num_branch = Dense(32, activation='relu')(num_branch)
city_branch = Embedding(input_dim=NUM_CITIES, output_dim=50, name='city_embedding')(input_city); city_branch = Flatten()(city_branch)
combined = Concatenate()([text_branch, num_branch, city_branch])
z = Dense(512, activation='relu')(combined); z = Dropout(0.5)(z)
z = Dense(256, activation='relu')(z); z = Dropout(0.5)(z)

# Trzy oddzielne głowice wyjściowe
output_district = Dense(NUM_DISTRICTS, activation='softmax', name='district_output')(z)
output_subdistrict = Dense(NUM_SUBDISTRICTS, activation='softmax', name='subdistrict_output')(z)
output_street = Dense(NUM_STREETS, activation='softmax', name='street_output')(z)

model = Model(inputs=[input_text, input_num, input_city], outputs=[output_district, output_subdistrict, output_street])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 20000)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ num_input (InputLayer)        │ (None, 3)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │       2,560,128 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │             256 │ num_input[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_embedding (Embedding)    │ (None, 1, 50)             │         276,800 │ city_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │           2,080 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 50)                │               0 │ city_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 210)               │               0 │ dropout[0][0],             │
│                               │                           │                 │ dense_2[0][0],             │
│                               │                           │                 │ flatten[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 512)               │         108,032 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 512)               │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 256)               │         131,328 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 256)               │               0 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ district_output (Dense)       │ (None, 38)                │           9,766 │ dropout_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ subdistrict_output (Dense)    │ (None, 2167)              │         556,919 │ dropout_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 12,281,537 (46.85 MB)

 Trainable params: 12,281,537 (46.85 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Instancje funkcji straty dla każdej głowicy
penalty_config = {'wrong_parent': 1.5}
district_loss = 'sparse_categorical_crossentropy' # Dzielnica nie ma rodzica w naszym kontekście
subdistrict_loss = HierarchicalLoss(hierarchy_map, penalty_config, name='subdistrict_loss')
street_loss = HierarchicalLoss(hierarchy_map, penalty_config, name='street_loss')

# Kompilacja modelu
model.compile(
    optimizer='adam',
    loss={
        'district_output': district_loss,
        'subdistrict_output': subdistrict_loss,
        'street_output': street_loss
    },
    loss_weights={'district_output': 1.0, 'subdistrict_output': 1.2, 'street_output': 1.5},
    metrics={'district_output': 'accuracy', 'subdistrict_output': 'accuracy', 'street_output': 'accuracy'}
)

# Callbacks
model_checkpoint = ModelCheckpoint(MODEL_PATH, monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

# Przygotowanie słowników danych
X_train_dict = {'text_input': X_train_text, 'num_input': X_train_num, 'city_input': X_train_city_mapped}
X_val_dict = {'text_input': X_val_text, 'num_input': X_val_num, 'city_input': X_val_city_mapped}
y_train_dict = {
    'district_output': y_district_train_mapped,
    'subdistrict_output': y_subdistrict_train_mapped,
    'street_output': y_street_train_mapped
}
y_val_dict = {
    'district_output': y_district_val_mapped,
    'subdistrict_output': y_subdistrict_val_mapped,
    'street_output': y_street_val_mapped
}

# Trening
history = model.fit(
    X_train_dict, y_train_dict,
    validation_data=(X_val_dict, y_val_dict),
    epochs=20, batch_size=128,
    callbacks=[model_checkpoint, early_stopping]
)

Epoch 1/20
6934/6935 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - district_output_accuracy: 0.7367 - district_output_loss: 0.8553 - loss: 26.5172 - street_output_accuracy: 0.0147 - street_output_loss: 12.3722 - subdistrict_output_accuracy: 0.0875 - subdistrict_output_loss: 5.9197
Epoch 1: val_loss improved from inf to 17.70551, saving model to artifacts_v11\best_location_model_v11.keras
6935/6935 ━━━━━━━━━━━━━━━━━━━━ 579s 83ms/step - district_output_accuracy: 0.7367 - district_output_loss: 0.8553 - loss: 26.5162 - street_output_accuracy: 0.0147 - street_output_loss: 12.3718 - subdistrict_output_accuracy: 0.0875 - subdistrict_output_loss: 5.9194 - val_district_output_accuracy: 0.8431 - val_district_output_loss: 0.4699 - val_loss: 17.7055 - val_street_output_accuracy: 0.0606 - val_street_output_loss: 8.7091 - val_subdistrict_output_accuracy: 0.1879 - val_subdistrict_output_loss: 3.4770
Epoch 2/20
6934/6935 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - district_output_accuracy: 0.8291 - district_output_loss

## Predykcja z Post-processingiem i Rozszerzona Ewaluacja (REKOMENDACJA #2 i #4)

**Zmiana:** Ta komórka implementuje kluczową logikę biznesową:
1.  **Post-processing:** Weryfikujemy, czy przewidziane lokalizacje są ze sobą spójne hierarchicznie. Jeśli nie, unieważniamy niższe poziomy (np. błędna ulica dla danej pod-dzielnicy).
2.  **Rozszerzona Ewaluacja:** Obliczamy dokładność dla każdego poziomu (dzielnica, pod-dzielnica, ulica) oddzielnie, a także łączną dokładność, aby lepiej zrozumieć wyniki.

In [6]:
print(f"Wczytywanie najlepszego modelu z: {MODEL_PATH}")
best_model = tf.keras.models.load_model(
    MODEL_PATH, 
    custom_objects={'HierarchicalLoss': HierarchicalLoss}
)
print("Model wczytany pomyślnie.")

# Wczytujemy mapy odwrotne
inv_district_map = {v: k for k, v in district_map.items()}
inv_subdistrict_map = {v: k for k, v in subdistrict_map.items()}
inv_street_map = {v: k for k, v in street_map.items()}

num_samples = 1000
indices = np.random.choice(range(len(y_district_val)), num_samples, replace=False)

results_data = []

print(f"Przeprowadzanie predykcji z post-processingiem na {num_samples} próbkach...")
for i in indices:
    # Przygotowanie wejścia
    original_city_id = X_val_city[i]
    mapped_city_id = city_map.get(original_city_id, 0)
    dense_vector_sample = X_val_text[i].toarray()
    input_sample = {
        'text_input': dense_vector_sample,
        'num_input': np.expand_dims(X_val_num[i], axis=0),
        'city_input': np.expand_dims(mapped_city_id, axis=0)
    }
    
    # Surowa predykcja modelu
    pred_district_probs, pred_subdistrict_probs, pred_street_probs = best_model.predict(input_sample, verbose=0)
    
    # Odmapowanie indeksów na oryginalne ID
    pred_district_id = inv_district_map.get(np.argmax(pred_district_probs[0]), 0)
    pred_subdistrict_id = inv_subdistrict_map.get(np.argmax(pred_subdistrict_probs[0]), 0)
    pred_street_id = inv_street_map.get(np.argmax(pred_street_probs[0]), 0)
    
    # --- POST-PROCESSING HIERARCHICZNY ---
    # Krok 1: Weryfikacja pod-dzielnicy względem dzielnicy
    if pred_subdistrict_id != 0:
        parent_of_subdistrict = hierarchy_map.get(pred_subdistrict_id, -1)
        if parent_of_subdistrict != pred_district_id:
            pred_subdistrict_id = 0 # Unieważnij pod-dzielnicę
            pred_street_id = 0      # Unieważnij też ulicę
            
    # Krok 2: Weryfikacja ulicy względem pod-dzielnicy (jeśli pod-dzielnica jest poprawna)
    if pred_street_id != 0:
        parent_of_street = hierarchy_map.get(pred_street_id, -1)
        if parent_of_street != pred_subdistrict_id:
            pred_street_id = 0 # Unieważnij tylko ulicę
    
    # Pobranie prawdziwych ID
    true_district_id = y_district_val[i]
    true_subdistrict_id = y_subdistrict_val[i]
    true_street_id = y_street_val[i]
    
    results_data.append({
        'True_District': true_district_id, 'Pred_District': pred_district_id,
        'True_SubDistrict': true_subdistrict_id, 'Pred_SubDistrict': pred_subdistrict_id,
        'True_Street': true_street_id, 'Pred_Street': pred_street_id
    })

df_results = pd.DataFrame(results_data)

# --- ROZSZERZONA EWALUACJA ---
df_results['Correct_District'] = (df_results['True_District'] == df_results['Pred_District'])
# Poprawna pod-dzielnica = (ID się zgadzają) LUB (oba są 'Brak', czyli 0)
df_results['Correct_SubDistrict'] = (df_results['True_SubDistrict'] == df_results['Pred_SubDistrict'])
df_results['Correct_Street'] = (df_results['True_Street'] == df_results['Pred_Street'])

# Ogólna poprawność - wszystkie poziomy muszą się zgadzać
df_results['Correct_Overall'] = df_results['Correct_District'] & df_results['Correct_SubDistrict'] & df_results['Correct_Street']

acc_district = df_results['Correct_District'].mean()
acc_subdistrict = df_results['Correct_SubDistrict'].mean()
acc_street = df_results['Correct_Street'].mean()
acc_overall = df_results['Correct_Overall'].mean()

print("\n--- Wyniki Ewaluacji ---")
print(f"Dokładność dla Dzielnic: {acc_district:.2%}")
print(f"Dokładność dla Pod-dzielnic: {acc_subdistrict:.2%}")
print(f"Dokładność dla Ulic: {acc_street:.2%}")
print(f"\nDokładność CAŁKOWITA (wszystkie poziomy poprawne): {acc_overall:.2%}")

# Wizualizacja
def highlight_correct(s):
    return ['background-color: #d4edda' if s.Correct_Overall else 'background-color: #f8d7da'] * len(s)

for col in ['True_District', 'Pred_District', 'True_SubDistrict', 'Pred_SubDistrict', 'True_Street', 'Pred_Street']:
    df_results[f'{col}_Name'] = df_results[col].apply(lambda x: id_to_name.get(x, 'Brak'))
    
styled_results = df_results.sample(min(20, len(df_results)), random_state=RANDOM_STATE).style.apply(highlight_correct, axis=1)
print("\n--- Losowe wyniki predykcji (Zielony = Poprawna, Czerwony = Błędna) ---")
display(styled_results)

Wczytywanie najlepszego modelu z: artifacts_v11\best_location_model_v11.keras
Model wczytany pomyślnie.
Przeprowadzanie predykcji z post-processingiem na 1000 próbkach...

--- Wyniki Ewaluacji ---
Dokładność dla Dzielnic: 91.40%
Dokładność dla Pod-dzielnic: 59.10%
Dokładność dla Ulic: 56.60%

Dokładność CAŁKOWITA (wszystkie poziomy poprawne): 38.80%

--- Losowe wyniki predykcji (Zielony = Poprawna, Czerwony = Błędna) ---


,True_District,Pred_District,True_SubDistrict,Pred_SubDistrict,True_Street,Pred_Street,Correct_District,Correct_SubDistrict,Correct_Street,Correct_Overall,True_District_Name,Pred_District_Name,True_SubDistrict_Name,Pred_SubDistrict_Name,True_Street_Name,Pred_Street_Name
521,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
737,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
740,3270,3270,72674,72674,0,349547,True,True,False,False,Wrocław-fabryczna,Wrocław-fabryczna,Fabryczna,Fabryczna,Brak,Legnicka
660,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
411,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
678,3265,3250,69149,97758,0,0,False,False,True,False,Łódź-bałuty,Łódź-polesie,Bałuty,Polesie,Brak,Brak
626,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
513,3275,3275,66540,66540,0,258701,True,True,False,False,Kraków-podgórze,Kraków-podgórze,Podgórze,Podgórze,Brak,Wielicka
859,3268,3268,76803,96402,0,0,True,False,True,False,Poznań-nowe miasto,Poznań-nowe miasto,Starołęka mała,Żegrze,Brak,Brak
136,0,0,105707,0,487553,0,True,False,False,False,Brak,Brak,Centrum,Brak,Biskupa czesława kaczmarka,Brak
